In [1]:
import psycopg2
import pandas as pd
import pandas.io.sql as pdsql
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 

from pandas.tools.plotting import scatter_matrix
from sklearn import cross_validation as cv
from sklearn.cross_validation import train_test_split as tts
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Ridge, RandomizedLasso, ElasticNet, LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

/Users/gbq483/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
con=psycopg2.connect(dbname= 'DCZillow', 
                     host='dczillow.cfdlhqngxmri.us-east-1.rds.amazonaws.com',
                     port='5432',
                     user= 'DCZillow', 
                     password= 'DCZillow');

cur = con.cursor()

## Pull Aggregated Basic Business License Change by Zipcode

In [3]:
df_lic_sum = pdsql.read_sql("""SELECT * FROM license_summary_float""", con)

df_lic_sum.loc[:,'bbl_pct_chg_1617'] = df_lic_sum['growth_yoy']
df_lic_sum.loc[:,'bbl_pct_chg_1217'] = df_lic_sum['growth_yoy_5']

df_lic_sum1 = df_lic_sum[['zipcode', 'bbl_pct_chg_1617', 'bbl_pct_chg_1217']]
df_lic_sum1

,zipcode,bbl_pct_chg_1617,bbl_pct_chg_1217
0,20001.0,-0.006413,13.084848
1,20002.0,0.052114,6.374251
2,20003.0,0.419821,4.894286
3,20004.0,0.294461,5.830769
4,20005.0,0.217814,11.965517
5,20006.0,0.194030,5.779661
6,20007.0,-0.092703,10.072993
7,20008.0,0.063433,11.127660
8,20009.0,0.677718,22.250000
9,20010.0,-0.161587,1.720867


## Pull Basic Business License Category Change by Zipcode

In [4]:
df_lic_cat_xtab_6m= pdsql.read_sql("""SELECT * FROM lic_cat_crosstab_6m_yoy""", con)

df_lic_cat_xtab_6m.fillna(0)

,zipcode,barber_shop,cigarette_retail,delicatessen,food_products,parking_facility_attendant,grocery_store,special_events,charitable_solicitation,home_improvement_salesman,...,parking_facility,gen_contr_construction_mngr,consumer_goods_auto_repair,apartment,two_family_rental,home_improvement_contractor,general_business_licenses,patent_medicine,restaurant,one_family_rental
0,20016,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,-0.500000,...,0.000000,0.272727,0.000000,-0.230769,0.000000,0.000000,-0.448560,0.000000,-0.795322,0.104396
1,20005,0.0,0.000000,0.000000,0.000000,2.727273,0.000000,0.000000,3.566038,0.000000,...,-0.083333,-0.866197,0.000000,0.000000,0.000000,0.000000,0.320069,-0.875000,-0.457831,0.000000
2,20036,0.0,0.000000,0.000000,0.000000,-0.080000,0.000000,0.000000,0.753623,0.000000,...,-0.333333,0.222222,0.000000,0.000000,0.000000,0.000000,0.558659,0.000000,-0.758929,0.172589
3,20037,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.333333,0.000000,0.000000,0.358209,0.000000,-0.023121,0.435583
4,20004,0.0,0.000000,2.666667,-0.571429,1.444444,0.000000,-0.571429,12.454545,0.000000,...,1.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,-0.400000,-0.306452,-0.419643
5,20007,0.0,0.000000,1.272727,-0.950704,0.000000,0.000000,0.000000,1.555556,0.000000,...,-0.333333,0.000000,0.000000,4.906250,0.000000,0.000000,0.946602,0.000000,-0.746544,0.000000
6,20017,0.0,0.000000,3.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,...,0.000000,0.000000,0.000000,1.107143,0.000000,0.000000,-0.637255,0.000000,0.000000,0.106557
7,20008,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019802,0.000000,-0.375000,0.117438
8,20002,0.0,-0.150000,0.377778,2.057692,0.000000,1.055556,0.000000,0.640000,-0.090909,...,0.000000,0.029412,0.111111,-0.323442,0.057895,0.000000,-0.247232,-0.791667,0.500000,0.238158
9,20003,0.0,-0.944444,0.000000,0.000000,1.000000,0.000000,-0.500000,0.000000,-0.125000,...,0.000000,0.071429,0.000000,2.784314,0.120000,0.000000,0.090909,0.000000,1.765625,0.646552


## Pull ZHVI Change by Zipcode

In [5]:
df_zhvi= pdsql.read_sql("""SELECT * FROM zhvi""", con)

columns = ('mom','qoq','yoy','fiveyear','tenyear','peakmonth','peakmonth','peakquarter','pctfallfrompeak')
col_prefix = 'zhvi_'

for i in columns:
    newvar = col_prefix + str(i)
    df_zhvi.rename(columns={i : newvar}, inplace=True)
    
df_zhvi.rename(columns={'regionname' : 'zipcode'}, inplace=True)
df_zhvi.loc[:,'zhvi_peakmonth'] = pd.to_datetime(df_zhvi['zhvi_peakmonth'])
df_zhvi.loc[:,'lasttimeatcurrzhvi'] = pd.to_datetime(df_zhvi['lasttimeatcurrzhvi'])

df_zhvi1 = df_zhvi[['zipcode', 'sizerank', 'zhvi','zhvi_mom','zhvi_qoq','zhvi_yoy','zhvi_fiveyear','zhvi_tenyear',
                    'zhvi_peakmonth','peakzhvi','zhvi_pctfallfrompeak','lasttimeatcurrzhvi']]
df_zhvi1.fillna(0)

,zipcode,sizerank,zhvi,zhvi_mom,zhvi_qoq,zhvi_yoy,zhvi_fiveyear,zhvi_tenyear,zhvi_peakmonth,peakzhvi,zhvi_pctfallfrompeak,lasttimeatcurrzhvi
0,20009,27,565400,-0.006152,-0.005977,0.064383,0.052053,0.024720,2017-03-01,572200,-0.011884,2017-02-01
1,20002,28,612700,-0.007291,-0.003091,0.048246,0.101941,0.000000,2017-03-01,619900,-0.011615,2017-02-01
2,20011,140,568900,-0.003852,0.001408,0.107671,0.123579,0.037577,2017-03-01,573000,-0.007155,2017-03-01
3,20019,205,293900,0.006507,0.021195,0.136944,0.119687,0.020936,2017-05-01,293900,0.000000,2017-05-01
4,20001,265,647600,-0.004764,-0.005681,0.065657,0.083636,0.040571,2017-03-01,653800,-0.009483,2017-02-01
5,20020,361,307800,-0.006135,-0.010608,0.081518,0.114632,0.018479,2017-03-01,311800,-0.012829,2017-01-01
6,20008,1225,856100,-0.004072,0.003634,0.054570,0.044144,0.018017,2017-03-01,860100,-0.004651,2017-03-01
7,20032,1576,278300,0.012368,0.032653,0.108323,0.100896,0.014032,2017-05-01,278300,0.000000,2017-05-01
8,20016,1883,983100,-0.004557,-0.003548,0.052119,0.052494,0.026411,2017-03-01,991200,-0.008172,2017-02-01
9,20003,2048,742500,-0.004158,-0.010528,0.061472,0.077594,0.036841,2017-03-01,751200,-0.011581,2017-02-01


In [4]:
test= pdsql.read_sql("""SELECT * FROM model_data_v1_1""", con)
test

,lastsolddate_6,bedrooms,bathrooms,yearbuilt,lotsizesqft,taxassessment_6,finishedsqft,for_sale,zpid,street,...,finishedsqft_missing,target_2017,Log_lotsizesqft,Log_taxassessment_6,built_after_2000,sqft_lt1000,sqft_lt1500,sqft_lt2000,sqft_lt2500,sqft_ge2500
